In [3]:
import os
import threading
from http.server import HTTPServer, SimpleHTTPRequestHandler
from local_packages import *
import subprocess
from dotenv import load_dotenv

# 加载环境变量
dotenv_path = os.path.join(os.getcwd(), '.env')

# 设置项目根目录和图片目录
project_root = os.path.dirname(dotenv_path)

service_type = 'zhipu'

def initialize_service(service_type):
    if service_type in ['zhipu', None]:
        version = 'glm-3-turbo'
        #'glm-4' 'glm-4v' 'glm-3-turbo'
        service = GLMService(version)
    elif service_type in ['kimi']:
        version = '8k'
        #'8k'1M/12￥ '32k'1M/24￥ '128k'1M/60￥
        service = KimiService(version)
    elif service_type in ['sensetime']:
        version = 'SenseChat'
        #SenseChat SenseChat-32K SenseChat-128K SenseChat-Turbo SenseChat-FunctionCall
        service = SenseService(version=version)
    else:
        raise ValueError('未知的服务类型')
    
    return service

service = initialize_service(service_type)

In [4]:
#pip install pylatex

In [5]:
import zipfile
from pylatex import Document, Section, Subsection, Tabular, Math, TikZ, Axis, \
    Plot, Figure, Matrix, Alignat
from pylatex.utils import italic
import os
import re

In [6]:
def createtex(name : str, folder_name : str, code_blocks : list)->None:
    # 合并文件夹路径
    folder_path = os.path.join(project_root, folder_name)
    # 检查文件夹是否存在
    if not os.path.exists(folder_path):
        # 如果文件夹不存在，则创建它
        os.makedirs(folder_path)
        print(f"文件夹 '{folder_name}' 已成功创建。")
    else:
        # 如果文件夹已存在，打印一条消息
        print(f"文件夹 '{folder_name}' 已存在。")
    # 创建一个新的.tex文件用于写入代码块
    tex_file_path = os.path.join(folder_path, name)
    with open(tex_file_path, 'w', encoding='utf-8') as tex_file:
        # 遍历所有匹配的代码块
        for code_block in code_blocks:
            # 将代码块写入.tex文件
            tex_file.write(code_block + '\n\n')
    # 打印消息表示文件已创建
    print("TeX文件已生成。")

def theme_to_latex(question : str) -> str:
    # 根据输入主题生成PPT    
    #——————————————————————————————
    # 主问答
    #——————————————————————————————
    
    # 定义内容，清晰说明任务目标和期望的输出格式
    input_text1 = f'根据主题{question}，查找其有关的历史背景和发展脉络，相关的主要理论和模型，包括它们的起源、主要观点，不要总结'
    round1 = service.ask_once(input_text1)

    input_text2 = f'根据主题{question}，概述该领域的关键研究和发现，与对该主题有重大贡献的学者和研究者，不要总结'
    round2 = service.ask_once(input_text2)

    input_text3 = f'根据主题{question}，探讨该主题的实际应用和实践，预测该主题未来的研究方向、潜在的发展趋势，不要总结'
    round3 = service.ask_once(input_text3)

    ask = f" 使用beamer库编写用于生成PPT的latex源码，使用'[UTF8] ctex'，内容须包括'{round1}''{round2}''{round3}',多分几页详细介绍一下,请返回一整段纯代码不要有多余的解释"
    answer = service.ask_once(ask)

    #——————————————————————————————
    # 解析模块
    #——————————————————————————————
    
    # 定义正则表达式模式，匹配```与```之间的内容
    code_pattern = re.compile(r'```latex(.*?)```', re.DOTALL)
    # 使用正则表达式找到所有匹配的代码块
    code_blocks = code_pattern.findall(answer)

    #——————————————————————————————
    # os模块
    #——————————————————————————————

    createtex('output.tex', "test", code_blocks)
    #——————————————————————————————
    # 编译模块
    #——————————————————————————————
    #subprocess.run(["pdflatex", "output.tex"])
    #os.chdir(current_directory)
    return answer


In [7]:
question='航空发动机'
answer = theme_to_latex(question)

print(answer)

文件夹 'test' 已成功创建。
TeX文件已生成。
以下是使用Beamer库编写的一个LaTeX文档，它将生成一个包含所提供内容的PPT。请确保您安装了CTeX发行版或类似的LaTeX发行版，并且您的LaTeX环境支持中文。

```latex
\documentclass{beamer}
\usepackage[UTF8]{ctex}
\usepackage{graphicx}
\usepackage{multirow}
\usepackage{caption}
\usepackage{subcaption}

\title{航空发动机的历史背景和发展脉络}
\author{作者名字}
\institute{机构名称}
\date{\today}

\begin{document}

\begin{frame}
  \titlepage
\end{frame}

\begin{frame}
  \frametitle{航空发动机的历史背景和发展脉络}
  \begin{itemize}
    \item 早期航空发动机：活塞发动机，1903年莱特兄弟首次飞行。
    \item 喷气发动机的诞生：20世纪40年代，喷气发动机的出现。
    \item 高性能发动机的发展：涡轮风扇发动机、涡轮喷气发动机、涡轮冲压发动机等。
  \end{itemize}
\end{frame}

\begin{frame}
  \frametitle{相关的主要理论和模型}
  \begin{itemize}
    \item 牛顿力学：飞行力学、气动力学、热力学。
    \item 气动力学：空气动力学设计、燃烧稳定性和排放控制。
    \item 热力学：燃料燃烧、热效率和温度控制。
    \item 控制理论：发动机控制、故障诊断和飞行管理系统。
  \end{itemize}
\end{frame}

\begin{frame}
  \frametitle{航空发动机领域的现状与挑战}
  \begin{itemize}
    \item 航空发动机是现代工业中最复杂的领域之一。
    \item 涉及到多学科和诸多尖端技术的集成。
    \item 数字化技术对航空发动机的全生命周期产生了深刻影响。
  \end{itemize}
\end{fra

In [9]:
# 解压缩 zip 文件，初始化一个变量zip_ref并把解压缩后的文件放到一个文件夹'unzipped_files'中，若没有该文件夹则创建一个
#def extract_zip_file(zip_file_path):
#    with zipfile.ZipFile(zip_file_path, mode='r') as zip_ref:
#        zip_ref.extractall('unzipped_files')
#    return 'unzipped_files'
    
#读取解压缩后文件夹的main.tex
def read_main_tex_file(main_tex_path):
    with open(main_tex_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

#梳理main.tex中的内容
def latex2ppt(file_content):
    
    #思维链逐步提问
    #先梳理一遍内容，看看要划分成几个大的部分
    input_text = f'{file_content}是一essay的latex源码，请你根据源码的标题与内容对内容结构进行梳理，目的是制作学术组会报告内容ppt，将内容分类为包括但不限于Introduction、Literature Review、Model、Numerical analysis、Background等等大类，根据需要自行发挥'
    round1 = service.ask_once(input_text)
    
    #根据round1梳理的框架结合源码内容指定组会汇报框架
    input_text = f'{file_content}是essay源码内容，{round1}是本次组会汇报大致的汇报框架，请你根据内容与汇报框架完善各部分汇报内容'
    round2 = service.ask_once(input_text)
    
    #生成beamer代码
    input_text = f'请你根据框架{round1}与内容{round2}，使用beamer库编写用于生成PPT的latex源码,请返回纯代码不要有多余的解释'
    round3 = service.ask_once(input_text)
    
    #——————————————————————————————
    # 解析模块
    #——————————————————————————————
    
    # 定义正则表达式模式，匹配```与```之间的内容
    code_pattern = re.compile(r'```latex(.*?)```', re.DOTALL)
    # 使用正则表达式找到所有匹配的代码块
    code_blocks = code_pattern.findall(round3)

    #——————————————————————————————
    # os模块
    #——————————————————————————————

    createtex('output.tex', "latex_to_ppt", code_blocks)
    return round3

#执行
#unzip_file_path = extract_zip_file('./概统大作业——动森大头菜买卖分析.zip')
folder_name='test'
name='output.tex'
folder_path = os.path.join(project_root, folder_name)
tex_file_path = os.path.join(folder_path, name)
main_tex_content = read_main_tex_file(tex_file_path)
beamer_code = latex2ppt(main_tex_content)
print("生产的beamer代码为:", beamer_code)


文件夹 'latex_to_ppt' 已成功创建。
TeX文件已生成。
生产的beamer代码为: 以下是一个基于Beamer库的LaTeX源码示例，用于生成包含上述报告内容的PPT幻灯片：

```latex
\documentclass{beamer}
\usepackage{ctex} % 适用于中文
\usetheme{Warsaw} % 选择主题，Warsaw是Beamer自带的一个主题

\title{航空发动机技术报告}
\author{报告人姓名}
\institute{报告人所在机构}
\date{\today}

\begin{document}

\begin{frame}
  \titlepage
\end{frame}

\section{引言}
\begin{frame}
  \frametitle{引言}
  \begin{itemize}
    \item 报告题目与主题介绍
    \item 航空发动机的重要性与地位
  \end{itemize}
\end{frame}

\section{文献综述}
\begin{frame}
  \frametitle{文献综述}
  \begin{itemize}
    \item 航空发动机的历史发展
    \item 相关主要理论与模型
  \end{itemize}
\end{frame}

\section{模型}
\begin{frame}
  \frametitle{模型}
  \begin{itemize}
    \item 设计与运作模型介绍
    \item 数字化技术的影响
  \end{itemize}
\end{frame}

\section{数值分析}
\begin{frame}
  \frametitle{数值分析}
  \begin{itemize}
    \item 现状与挑战分析
    \item 多学科集成与数字化影响
  \end{itemize}
\end{frame}

\section{背景}
\begin{frame}
  \frametitle{背景}
  \begin{itemize}
    \item 院士团队的组织策略
    \item 智能化技术进展与关键概述
    \item 数字化试验理论和技术路径
  \e